In [81]:
import random
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [82]:
def load_tutors_data(TUTOR_PATH):
    csv_path = os.path.join(TUTOR_PATH)
    return pd.read_csv(csv_path)

In [83]:
tutors_train = load_tutors_data("train.csv")
tutors_test = load_tutors_data("test.csv")

In [84]:
tutors_train_copy = tutors_train.copy()
tutors_test_copy = tutors_test.copy()

In [85]:
tutors_train_copy.head()

,Id,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history,mean_exam_points,choose
0,0,35.0,0.0,2150.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,74.0,0
1,1,52.0,2.0,1250.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,57.0,1
2,2,29.0,3.0,1750.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,66.0,0
3,3,33.0,3.0,1050.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,66.0,1
4,4,46.0,3.0,2250.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,73.0,0


In [86]:
tutors_train_copy = tutors_train_copy.drop("Id", axis=1)
tutors_test_copy = tutors_test_copy.drop("Id", axis=1)

In [87]:
target = tutors_train_copy['choose']

In [88]:
tutors_train_copy = tutors_train_copy.drop("choose", axis=1)

In [89]:
tutors_train_copy.head()

,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history,mean_exam_points
0,35.0,0.0,2150.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,74.0
1,52.0,2.0,1250.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,57.0
2,29.0,3.0,1750.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,66.0
3,33.0,3.0,1050.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,66.0
4,46.0,3.0,2250.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,73.0


In [90]:
X_train, X_test, y_train, y_test = train_test_split(tutors_train_copy, target, test_size=0.3, random_state=42)

In [91]:
# Добавлен коэффициент b0 в алгоритм предсказывания

def gb_predict(X, trees_list, coef_list, eta, b0=0):
    return np.array([sum([b0 + eta* coef * alg.predict([x])[0] for alg, coef in zip(trees_list, coef_list)]) for x in X])

In [92]:
random.seed(42)

def get_bootstrap(data, labels, N):
    n_samples = data.shape[0]
    bootstrap = []
    
    for i in range(N):
        b_data = np.zeros(data.shape)
        b_labels = np.zeros(labels.shape)
        
        for j in range(n_samples):
            sample_index = random.randint(0, n_samples-1)
            b_data[j] = data.iloc[sample_index]
            b_labels[j] = labels.iloc[sample_index]
        bootstrap.append((b_data, b_labels))
        
    return bootstrap

In [93]:
def get_subsample(len_sample):
    # будем сохранять не сами признаки, а их индексы
    sample_indexes = [i for i in range(len_sample)]
    
    len_subsample = int(np.sqrt(len_sample))
    subsample = []
    
    random.shuffle(sample_indexes)
    for _ in range(len_subsample):
        subsample.append(sample_indexes.pop())
        
    return subsample

In [94]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [95]:
class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        # подсчет количества объектов разных классов
        classes = {}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        #  найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
        prediction = max(classes, key=classes.get)
        return prediction        

In [96]:
def calc_entropy(labels):
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    impurity = 0
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p * np.log2(p)
        
    return impurity

In [97]:
def quality(left_labels, right_labels, current_var):

    # доля выбоки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return current_var - p * calc_entropy(left_labels) - (1 - p) * calc_entropy(right_labels)

In [98]:
def split(data, labels, index, t):
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [99]:
def get_values(t_val, n):
    t_val = np.sort(t_val)
    new_value = []
    temp = []
    for i, item in enumerate(t_val):
        if (i%n == 0) or (i == len(t_val) - 1):
            new_value.append(np.mean(temp))
            temp = []
        else:
            temp.append(item)

    return new_value     

In [100]:
def find_best_split(data, labels):
    
    #  обозначим минимальное количество объектов в узле
    min_leaf = 4

    current_var = calc_entropy(labels)

    best_quality = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    # выбор индекса из подвыборки длиной sqrt(n_features)
    subsample = get_subsample(n_features)
    
    for index in subsample:
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = [row[index] for row in data]
        
        # выбор t_value, исходя из средних значений подвыборок
        for line in range(10, 511, 50):
            t_values = get_values(t_values, line)
        
            for t in t_values:
                true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
                #  пропускаем разбиения, в которых в узле остается менее 5 объектов
                if len(true_data) < min_leaf or len(false_data) < min_leaf:
                    continue
            
                current_quality = quality(true_labels, false_labels, current_var)
            
                #  выбираем порог, на котором получается максимальный прирост качества
                if current_quality > best_quality:
                    best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

In [101]:
def build_tree(data, labels):

    quality, t, index = find_best_split(data, labels)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if quality == 0:
        return Leaf(data, labels)
    

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels)
    false_branch = build_tree(false_data, false_labels)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [102]:
def random_forest(data, labels, n_trees):
    forest = []
    bootstrap = get_bootstrap(data, labels, n_trees)
    
    for b_data, b_labels in bootstrap:
        forest.append(build_tree(b_data, b_labels))
        
    return forest

In [103]:
def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [107]:
def predict(data, tree):
    
    classes = []
    for obj in range(data.shape[0]):
        prediction = classify_object(data.iloc[obj], tree)
        classes.append(prediction)
    return classes

In [104]:
def tree_vote(forest, data):

    # добавим предсказания всех деревьев в список
    predictions = []
    for tree in forest:
        predictions.append(predict(data, tree))
    
    # сформируем список с предсказаниями для каждого объекта
    predictions_per_object = list(zip(*predictions))
    
    # выберем в качестве итогового предсказания для каждого объекта то,
    # за которое проголосовало большинство деревьев
    voted_predictions = []
    for obj in predictions_per_object:
        voted_predictions.append(np.mean(obj))
        
    return voted_predictions

In [105]:
n_trees = 35

my_forest_1 = random_forest(X_train, y_train, n_trees)

D:\anaconda3\envs\machine_learning\lib\site-packages\numpy\core\fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
D:\anaconda3\envs\machine_learning\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
D:\anaconda3\envs\machine_learning\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in less_equal
  This is separate from the ipykernel package so we can avoid doing imports until
D:\anaconda3\envs\machine_learning\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in greater
  after removing the cwd from sys.path.


In [108]:
train_answers = tree_vote(my_forest_1, X_test)

In [109]:
from sklearn.metrics import roc_auc_score

In [110]:
roc_auc_score(y_test, train_answers)

0.8009711447534322

In [111]:
train_answers = tree_vote(my_forest_1, tutors_test_copy)

In [116]:
df_test_2 = pd.DataFrame(train_answers, columns=['choose'])
df_test_2

,choose
0,0.000000
1,0.314286
2,0.000000
3,0.000000
4,0.000000
...,...
9995,0.028571
9996,0.057143
9997,0.228571
9998,0.800000


In [117]:
df_test_1 = pd.read_csv("submission_example.csv", delimiter=',')
df_test_1['Id']

0       10000
1       10001
2       10002
3       10003
4       10004
        ...  
9995    19995
9996    19996
9997    19997
9998    19998
9999    19999
Name: Id, Length: 10000, dtype: int64

In [119]:
df_test = pd.concat([df_test_1['Id'], df_test_2['choose']], axis=1)
df_test.to_csv('submission.csv', index=False)